## 1. 数据预处理

### 1.1 根据股票代码划分数据
* 训练集

In [1]:
import pandas as pd

df = pd.read_csv("./data/train.csv")
stocks_code = df["kdcode"].unique()

print(stocks_code)
len(stocks_code)

['000001.SZ' '000157.SZ' '000333.SZ' '000568.SZ' '000703.SZ' '000768.SZ'
 '002024.SZ' '002044.SZ' '002049.SZ' '002120.SZ' '002230.SZ' '002271.SZ'
 '002311.SZ' '002371.SZ' '002456.SZ' '002602.SZ' '002607.SZ' '002714.SZ'
 '002773.SZ' '300003.SZ' '300033.SZ' '300124.SZ' '300144.SZ' '300628.SZ'
 '600000.SH' '600009.SH' '600016.SH' '600019.SH' '600025.SH' '600028.SH'
 '600030.SH' '600031.SH' '600036.SH' '600048.SH' '600050.SH' '600061.SH'
 '600104.SH' '600115.SH' '600196.SH' '600276.SH' '600309.SH' '600340.SH'
 '600383.SH' '600489.SH' '600519.SH' '600547.SH' '600570.SH' '600585.SH'
 '600588.SH' '600690.SH' '600703.SH' '600745.SH' '600809.SH' '600837.SH'
 '600848.SH' '600886.SH' '600887.SH' '600926.SH' '600958.SH' '601006.SH'
 '601088.SH' '601166.SH' '601169.SH' '601238.SH' '601318.SH' '601319.SH'
 '601336.SH' '601398.SH' '601555.SH' '601601.SH' '601628.SH' '601788.SH'
 '601838.SH' '601857.SH' '601872.SH' '601899.SH' '601919.SH' '601990.SH'
 '601998.SH' '603233.SH' '603799.SH' '603833.SH']


82

In [2]:
# 根据股票代码划分数据
for i, stock_i in enumerate(stocks_code):
    stock_i_data = df[df['kdcode'].isin([stock_i])]
    exec("train_df%s = stock_i_data"%i)

print("train_df0:")
print(train_df0.head())
print("###########################")
print("train_df81:")
print(train_df81.head())

for i in range(82):
    exec("print(train_df%s.shape)" % i)

train_df0:
      kdcode          dt    volume  turnover  adj_open  adj_high   adj_low  \
0  000001.SZ  2016-01-04  0.053897  0.029403  0.106737  0.106083  0.100116   
1  000001.SZ  2016-01-05  0.063440  0.033641  0.100229  0.102017  0.099401   
2  000001.SZ  2016-01-06  0.049326  0.026345  0.101566  0.101928  0.101546   
3  000001.SZ  2016-01-07  0.016715  0.008673  0.101477  0.100603  0.097256   
4  000001.SZ  2016-01-08  0.071499  0.037016  0.099694  0.099542  0.097167   

   adj_downprice  adj_upprice   pct_chg  adj_close  
0       0.106623     0.106641  0.084445   0.100536  
1       0.100793     0.100739  0.197473   0.101159  
2       0.101386     0.101386  0.207262   0.102315  
3       0.102572     0.102518  0.091647   0.097067  
4       0.097334     0.097263  0.216496   0.098668  
###########################
train_df81:
          kdcode          dt    volume  turnover  adj_open  adj_high  \
80853  603833.SH  2017-03-28  0.000012  0.000038  0.007539  0.007467   
80854  603833.SH  

* 测试集

In [3]:
df2 = pd.read_csv("./data/test.csv")
stocks_code2 = df2["kdcode"].unique()

# 根据股票代码划分数据
for i, stock_i in enumerate(stocks_code2):
    stock_i_data = df2[df2['kdcode'].isin([stock_i])]
    exec("test_df%s = stock_i_data" % i)

# print("test_df0:")
# print(test_df0.head())
# print("###########################")
# print("test_df81:")
# print(test_df81.head())

# for i in range(82):
#     exec("print(test_df%s.shape)" % i)

共82支股票

训练集按股票分为`train_df0`~`train_df81`

测试集按股票分为`test_df0`~`test_df81`

## 2. 

In [ ]:
feanum=9 #一共有多少特征
window=5 #时间窗设置